In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [10]:
## Reading Data
df_quotient = pd.read_csv("/drive/My Drive/Colab Notebooks/QUOTIENT_RAW.csv")

In [11]:
df_quotient.head()

,Unnamed: 0,Geschlecht,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,pl1,pl2,pl3,pl4,pl5,pl6,pl7,pl8,pl9,pl10,pl11,pl12,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12
0,0,1,5,2,5,5,4,2,4,6,2,6,2,5,2,2,5,1,2,2,2,2,6,2,2,5,5,3,2,2,5,4,2,3,1,3,4,2
1,1,2,4,2,4,3,2,3,1,3,4,4,3,3,2,3,4,2,2,1,2,2,5,1,1,4,3,2,3,1,5,5,3,3,2,4,1,4
2,2,1,3,2,3,4,3,2,4,3,2,5,3,4,3,2,4,2,2,3,2,4,4,2,3,4,3,2,1,2,4,5,2,2,2,5,4,2
3,3,2,5,2,6,6,5,2,4,5,3,5,3,5,1,4,5,1,2,2,3,2,5,1,1,4,5,3,3,3,5,3,2,4,3,4,4,3
4,4,1,3,5,4,2,1,4,1,3,3,4,4,2,2,5,3,2,3,2,2,2,4,3,3,4,2,4,1,2,3,3,3,2,2,2,3,2


## Processing Data
Based on this paper (https://www.researchgate.net/publication/6651835_The_Broad_Autism_Phenotype_Questionnaire), there are some questions that must be reversed, we will prepared it first

In [12]:
cols_to_reversed = ['a1','a3','a4','a5','a7','a8','a10','a12',
                    'pl3','pl9','pl12','r1','r5','r6','r10']

In [14]:
for cols in cols_to_reversed:
  df_quotient[cols] = df_quotient[cols].apply(lambda x: 1 if x==6 else 2 if x==5 else 3 if x==4
                                     else 4 if x==3 else 5 if x==2 else 6)

In [16]:
## calculate the scoring personality
df_quotient['Aloof Score'] = df_quotient[['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12']].mean(axis=1)
df_quotient['Pragmatic Score'] = df_quotient[['pl1','pl2','pl3','pl4','pl5','pl6','pl7','pl8','pl9','pl10','pl11','pl12']].mean(axis=1)
df_quotient['Rigid Score'] = df_quotient[['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','r11','r12']].mean(axis=1)

In [39]:
#making the label
score = np.array(df_quotient[['Aloof Score','Pragmatic Score','Rigid Score']])
label = []
for i in range(len(score)):
  result = score[i].argmax(axis=0)
  label.append(result)

In [41]:
df_quotient['Label'] = pd.DataFrame(label)
df_quotient.head()

,Unnamed: 0,Geschlecht,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,pl1,pl2,pl3,pl4,pl5,pl6,pl7,pl8,pl9,pl10,pl11,pl12,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,Aloof Score,Pragmatic Score,Rigid Score,Label
0,0,1,2,2,2,2,3,2,3,1,2,1,2,2,2,2,2,1,2,2,2,2,1,2,2,2,2,3,2,2,2,3,2,3,1,4,4,2,2.000000,1.833333,2.500000,2
1,1,2,3,2,3,4,5,3,6,4,4,3,3,4,2,3,3,2,2,1,2,2,2,1,1,3,4,2,3,1,2,2,3,3,2,3,1,4,3.666667,2.000000,2.500000,0
2,2,1,4,2,4,3,4,2,3,4,2,2,3,3,3,2,3,2,2,3,2,4,3,2,3,3,4,2,1,2,3,2,2,2,2,2,4,2,3.000000,2.666667,2.333333,0
3,3,2,2,2,1,1,2,2,3,2,3,2,3,2,1,4,2,1,2,2,3,2,2,1,1,3,2,3,3,3,2,4,2,4,3,3,4,3,2.083333,2.000000,3.000000,2
4,4,1,4,5,3,5,6,4,6,4,3,3,4,5,2,5,4,2,3,2,2,2,3,3,3,3,5,4,1,2,4,4,3,2,2,5,3,2,4.333333,2.833333,3.083333,0


In [44]:
## encode gender
df_quotient["Geschlecht"].value_counts()
df_quotient_filtered = df_quotient[df_quotient['Geschlecht']!=3]
df_quotient_filtered = df_quotient_filtered.reset_index(drop=True)
df_quotient_filtered['Gender'] = df_quotient_filtered['Geschlecht'].apply(lambda x: "Male" if x==1 else "Female")

In [46]:
#seperating features and label
df_features = df_quotient_filtered.drop(columns=['Unnamed: 0','Geschlecht','Aloof Score',	'Pragmatic Score',	'Rigid Score', 'Label'],axis=1)
df_label = df_quotient_filtered['Label']

In [48]:
##Scaling Answer
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12',
             'pl1','pl2','pl3','pl4','pl5','pl6','pl7','pl8','pl9','pl10','pl11','pl12',
             'r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','r11','r12']

features_minmax_transform = pd.DataFrame(data = df_features)
features_minmax_transform[numerical] = scaler.fit_transform(df_features[numerical])
features_minmax_transform.head()

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,pl1,pl2,pl3,pl4,pl5,pl6,pl7,pl8,pl9,pl10,pl11,pl12,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,Gender
0,0.2,0.2,0.2,0.25,0.4,0.2,0.4,0.0,0.2,0.00,0.2,0.2,0.2,0.2,0.25,0.0,0.2,0.25,0.2,0.2,0.00,0.25,0.2,0.2,0.2,0.4,0.2,0.2,0.2,0.4,0.2,0.4,0.0,0.6,0.6,0.2,Male
1,0.4,0.2,0.4,0.75,0.8,0.4,1.0,0.6,0.6,0.50,0.4,0.6,0.2,0.4,0.50,0.2,0.2,0.00,0.2,0.2,0.25,0.00,0.0,0.4,0.6,0.2,0.4,0.0,0.2,0.2,0.4,0.4,0.2,0.4,0.0,0.6,Female
2,0.6,0.2,0.6,0.50,0.6,0.2,0.4,0.6,0.2,0.25,0.4,0.4,0.4,0.2,0.50,0.2,0.2,0.50,0.2,0.6,0.50,0.25,0.4,0.4,0.6,0.2,0.0,0.2,0.4,0.2,0.2,0.2,0.2,0.2,0.6,0.2,Male
3,0.2,0.2,0.0,0.00,0.2,0.2,0.4,0.2,0.4,0.25,0.4,0.2,0.0,0.6,0.25,0.0,0.2,0.25,0.4,0.2,0.25,0.00,0.0,0.4,0.2,0.4,0.4,0.4,0.2,0.6,0.2,0.6,0.4,0.4,0.6,0.4,Female
4,0.6,0.8,0.4,1.00,1.0,0.6,1.0,0.6,0.4,0.50,0.6,0.8,0.2,0.8,0.75,0.2,0.4,0.25,0.2,0.2,0.50,0.50,0.4,0.4,0.8,0.6,0.0,0.2,0.6,0.6,0.4,0.2,0.2,0.8,0.4,0.2,Male


In [49]:
# encode Gender
features_final = pd.get_dummies(features_minmax_transform)
features_final.head()

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,pl1,pl2,pl3,pl4,pl5,pl6,pl7,pl8,pl9,pl10,pl11,pl12,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,Gender_Female,Gender_Male
0,0.2,0.2,0.2,0.25,0.4,0.2,0.4,0.0,0.2,0.00,0.2,0.2,0.2,0.2,0.25,0.0,0.2,0.25,0.2,0.2,0.00,0.25,0.2,0.2,0.2,0.4,0.2,0.2,0.2,0.4,0.2,0.4,0.0,0.6,0.6,0.2,0,1
1,0.4,0.2,0.4,0.75,0.8,0.4,1.0,0.6,0.6,0.50,0.4,0.6,0.2,0.4,0.50,0.2,0.2,0.00,0.2,0.2,0.25,0.00,0.0,0.4,0.6,0.2,0.4,0.0,0.2,0.2,0.4,0.4,0.2,0.4,0.0,0.6,1,0
2,0.6,0.2,0.6,0.50,0.6,0.2,0.4,0.6,0.2,0.25,0.4,0.4,0.4,0.2,0.50,0.2,0.2,0.50,0.2,0.6,0.50,0.25,0.4,0.4,0.6,0.2,0.0,0.2,0.4,0.2,0.2,0.2,0.2,0.2,0.6,0.2,0,1
3,0.2,0.2,0.0,0.00,0.2,0.2,0.4,0.2,0.4,0.25,0.4,0.2,0.0,0.6,0.25,0.0,0.2,0.25,0.4,0.2,0.25,0.00,0.0,0.4,0.2,0.4,0.4,0.4,0.2,0.6,0.2,0.6,0.4,0.4,0.6,0.4,1,0
4,0.6,0.8,0.4,1.00,1.0,0.6,1.0,0.6,0.4,0.50,0.6,0.8,0.2,0.8,0.75,0.2,0.4,0.25,0.2,0.2,0.50,0.50,0.4,0.4,0.8,0.6,0.0,0.2,0.6,0.6,0.4,0.2,0.2,0.8,0.4,0.2,0,1


## Modelling

In [50]:
#Splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_final, df_label, train_size=0.80, random_state=1)
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 196 samples.
Testing set has 50 samples.


In [51]:
## SVM Modelling

from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
def f_beta_score(y_true, y_predict):
    return fbeta_score(y_true, y_predict, beta = 0.5)

In [59]:
clf = SVC(random_state=1,decision_function_shape='ovo')
parameters = {'C':range(1,6),'kernel':['linear','poly','rbf','sigmoid'],'degree':range(1,6)}
# Make an fbeta_score scoring object using make_scorer()
#scorer = make_scorer(f_beta_score)
# Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(estimator = clf, param_grid = parameters)
# Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train, y_train)
# Get the estimator
best_clf = grid_fit.best_estimator_
# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)
# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
#print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
#print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
print("Final Parameter: ",best_clf)

Unoptimized model
------
Accuracy score on testing data: 0.7000

Optimized Model
------
Final accuracy score on the testing data: 0.7800
Final Parameter:  SVC(C=3, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=1, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)


In [60]:
clf = SVC(random_state=1,decision_function_shape='ovo',C=3,kernel='poly',degree=1)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
predictions

array([2, 0, 0, 2, 1, 1, 2, 0, 2, 0, 2, 0, 0, 1, 2, 0, 0, 2, 2, 2, 0, 2,
       0, 2, 2, 0, 1, 1, 2, 0, 0, 1, 0, 0, 2, 1, 1, 0, 2, 0, 2, 0, 2, 0,
       0, 0, 0, 0, 0, 2])

In [61]:
np.array(y_test)

array([2, 1, 0, 2, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 2, 0, 0, 2, 2, 2, 0, 2,
       0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 2, 1, 2, 0, 2, 2, 2, 0, 2, 0,
       0, 0, 0, 0, 0, 2])